In [ ]:
from wave_client import WaveClient
import pandas as pd
from utils import (
    load_environment_variables,
)


In [2]:
# Load environment variables and connect to WAVE backend
RESEARCHER_API_KEY, EXPERIMENTEE_API_KEY, WAVE_BACKEND_URL = load_environment_variables()

print(f"🔍 Connecting to WAVE backend...")
print(f"📡 Backend URL: {WAVE_BACKEND_URL}")
print(f"🔑 Using Researcher API key ending in: ...{RESEARCHER_API_KEY[-4:]}")

client = WaveClient(api_key=RESEARCHER_API_KEY, base_url=WAVE_BACKEND_URL)

Present working directory: c:\Users\inkpe\local-JS-Williams\epCD\tools
Loading environment variables from c:\Users\inkpe\local-JS-Williams\epCD\tools/.env
🔍 Connecting to WAVE backend...
📡 Backend URL: https://wave-backend-production-8781.up.railway.app
🔑 Using Researcher API key ending in: ...SYuP


In [ ]:
## use this (uncomment it) to see what commands exist 
## (allow the scrollable option at the bottom to see all commands)

# help(client.experiments)
# help(client.experiment_data)

In [3]:
# Get all experiments to see which one to analyze
experiments = await client.experiments.list(skip=0, limit=200)
print(f"Found {len(experiments)} experiments\n")

# Sort by creation date (most recent first) and show up to 5
sorted_experiments = sorted(experiments, key=lambda x: x.get('created_at', ''), reverse=True)[:5]
print(f"Showing {len(sorted_experiments)} most recent experiments (of {len(experiments)} total)\n")

# Display experiment info
for i, exp in enumerate(sorted_experiments, 1):
    print(f"{i}. Experiment UUID: {exp['uuid']}")
    print(f"   Description: {exp.get('description', 'N/A')}")
    print(f"   Type ID: {exp.get('experiment_type_id', 'N/A')}")
    print(f"   Created: {exp.get('created_at', 'N/A')[:10]}")
    print(f"   Tags: {', '.join(exp.get('tags', []))}")
    print("-" * 70)


Found 7 experiments

Showing 5 most recent experiments (of 7 total)

1. Experiment UUID: 6023b648-b34a-4a61-89e8-2589fb1b6e39
   Description: epcd_cup_prod
   Type ID: 15
   Created: 2026-02-12
   Tags: 
----------------------------------------------------------------------
2. Experiment UUID: 17a4a214-e500-4c90-985c-7d1fce2ddcae
   Description: Test experiment for epcd_cup - TEST_epcd_cup_20260212_004750
   Type ID: 15
   Created: 2026-02-12
   Tags: test
----------------------------------------------------------------------
3. Experiment UUID: 4a1d1324-64ab-464f-9787-b19058002f11
   Description: shadows_afc_shortnum_prod
   Type ID: 13
   Created: 2026-02-05
   Tags: 
----------------------------------------------------------------------
4. Experiment UUID: aeb293f8-2206-46a9-9d46-b21092526504
   Description: shadows_afc_numtask_prod
   Type ID: 12
   Created: 2026-02-05
   Tags: 
----------------------------------------------------------------------
5. Experiment UUID: 530a5fd4-2485

In [4]:
# Put the necessary info into the variables below 
selected_experiment_uuid = "6023b648-b34a-4a61-89e8-2589fb1b6e39" 

# Get all data from that experiment
print(f"\n📊 Fetching data for experiment {selected_experiment_uuid}...")
experiment_data_df = await client.experiment_data.get_all_data(
    experiment_id=selected_experiment_uuid
)

print(f"Total records: {len(experiment_data_df)}")
# print(experiment_data_df)



📊 Fetching data for experiment 6023b648-b34a-4a61-89e8-2589fb1b6e39...
Total records: 5413


In [5]:
# Filter and save into .csv file
full_df = (experiment_data_df.query("participant_id != '{{%PROLIFIC_PID%}}'")
           .sort_values(by="timestamp",ascending=True))
full_df.to_csv("data/dataframe_full.csv", index = False)

answer_df = full_df.query("trial_category == 'answerexpt'")
answer_df.to_csv("data/dataframe_answer.csv", index = False)



In [ ]:
# For creating new columns based on the information in firstStim & secondStim

stimColumns = ["firstStim","secondStim"]

for column in stimColumns:
    selected_column = answer_df[column]
    position_column = []

    for item in selected_column:
        if "Center" in item: 
            position_column.append("Center")
        elif "Right" in item:
            position_column.append("Right")
        else:
            position_column.append("Error")
    
    answer_df[column+"_cupPos"] = position_column
    answer_df.to_csv("data/dataframe_answer.csv", index = False)


